In [13]:
# cno_optimization.py
import cvxpy as cp
import numpy as np

def solve_cno_problem():
    # Network has 5 nodes: 0 through 4
    
    # Integer indices for the charger locations
    q1_idx = cp.Variable(integer=True)
    q2_idx = cp.Variable(integer=True)


    # Modified cost to encourage spread between q1 and q2
    # Encourage larger spread: penalize closeness → reward larger distance
    cost = (q1_idx + q2_idx) - 1.5 * (q2_idx - q1_idx)

    # Constraint
    constraints = [
        q1_idx >= 0, q1_idx <= 4, # Charger place on one of the 5 nodes
        q2_idx >= 0, q2_idx <= 4, # Charger place on one of the 5 nodes
        # cost >= 0, # Disallow cost to be negative
        cost >= np.clip(0, -1000, 0), # Clip negative costs to 0
        cost <= 7, # Chargers 1 and 2 cannot be at same location
        q1_idx <= q2_idx  # enforce ordering: q1 <= q2
    ]

    # Define price pi = (3-i)*qi_idx
    p1 = (3-1)*q1_idx
    p2 = (3-2)*q2_idx

    # Simple revenue model: total of active prices
    revenue = p1 + p2
    profit = revenue - cost
    constraints.append(profit >= 0)  # Ensure profit is non-negative

    # Define and solve the optimization problem
    objective = cp.Maximize(profit)
    prob = cp.Problem(objective, constraints)
    prob.solve()

    print("Optimization status:", prob.status)
    print("q1_idx + q2_idx:", q1_idx.value + q2_idx.value)
    print("cost (encouraging spread):", cost.value)
    print("revenue:", revenue.value)
    print("profit:", profit.value)

    return int(q1_idx.value), int(q2_idx.value), prob

In [14]:
q1_idx, q2_idx, prob = solve_cno_problem()
q1_idx, q2_idx, prob

Optimization status: optimal
q1_idx + q2_idx: 5.0
cost (encouraging spread): 0.5
revenue: 6.0
profit: 5.5


(1,
 4,
 Problem(Maximize(Expression(AFFINE, UNKNOWN, ())), [Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Variable((), var4597, integer=True)), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Variable((), var4598, integer=True)), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Variable((), var4597, integer=True)), Inequality(Constant(CONSTANT, ZERO, ()))]))

In [15]:
prob.solution

Solution(optimal, {4597: array(1.), 4598: array(4.)}, {}, {'solver_specific_stats': {'mip_gap': 0.0, 'mip_node_count': 1, 'mip_dual_bound': -5.5}})

In [16]:
# import cvxpy as cp
# import numpy as np
# from IPython.display import display

# def solve_vehicle_problem_binary_position(q1_idx, q2_idx):
#     T = 6  # Total time steps
#     n_nodes = 5

#     # State variables
#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)

#     # Binary variables
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     # done = cp.Variable(T, boolean=True)

    
#     # One-hot position encoding: pos_bin[t][i] = 1 if at node i at time t
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = []

#     # Initial conditions
#     constraints += [soc[0] == 6]
#     constraints += [budget[0] == 12]
#     constraints += [pos_bin[0, 0] == 1]  # Start at node 0
#     for i in range(1, n_nodes):
#         constraints += [pos_bin[0, i] == 0]

#     R_bar = 6  # Charging cost
    
#     node_indices = np.arange(n_nodes)  # [0, 1, 2, 3, 4]

#     for t in range(T):
#         # Ensure exactly one position at each time
#         constraints += [cp.sum(pos_bin[t, :]) == 1]
#         constraints += [cp.sum(pos_bin[t + 1, :]) == 1]

#         # Position at next timestep must be greater than or equal to current timestep

#         constraints += [(pos_bin[t + 1, :] @ node_indices) >= (pos_bin[t, :] @ node_indices)]
#         # constraints += [cp.sum(pos_bin[t + 1, :]) == 0]

#         # Movement constraints: only forward by 1
#         for i in range(n_nodes - 1):
#             constraints += [
#                 pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
#                 pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0
#             ]
#         # Ensure move from node 4 is not allowed
#         constraints += [pos_bin[t, 4] + move[t] <= 1]
        

#         # Dynamics
#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
#         # constraints += [move[t] + charge[t] + done[i] == 1]
#         constraints += [move[t] + charge[t] == 1]

#         constraints += [soc[t + 1] >= 4]
#         constraints += [budget[t + 1] >= 0]

#         # Charging only at charger nodes (q1 and q2)
#         constraints += [charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx]]

#     # Final conditions
#     constraints += [pos_bin[T, 4] == 1]  # Must end at node 4
#     constraints += [soc[T] >= 5]

#     # Define and solve problem
#     objective = cp.Minimize(cp.sum(charge))
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None, None, None, None, None

#     result = {
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None]
#     }

#     import pandas as pd
#     df = pd.DataFrame(result).set_index("t")
#     display("Vehicle State Trajectory", df)
#     # return soc.value, budget.value, move.value, charge.value, pos_bin.value
#     return problem


In [17]:
# print(f"(q1_idx: {q1_idx}, q2_idx: {q2_idx})")
# problem = solve_vehicle_problem_binary_position(q1_idx, q2_idx)
# problem

# # Function does not work for optimizations that terminate in less than T+1 timesteps

In [18]:
# print(f"(q1_idx: {q1_idx}, q2_idx: {q2_idx})")
# solve_vehicle_problem_binary_position(0, 4)

In [51]:
""" Code needs to be fixed. Having an error in the logic that is causing the vehicle to move forward when it charges."""

import cvxpy as cp
import numpy as np
import pandas as pd
from IPython.display import display

def solve_vehicle_problem_with_idle_and_done(q1_idx, q2_idx):
    T = 6  # Total time steps
    n_nodes = 5
    M = 1000  # Big-M constant

    # State variables
    soc = cp.Variable(T + 1)
    budget = cp.Variable(T + 1)

    # Binary action variables
    move = cp.Variable(T, boolean=True)
    charge = cp.Variable(T, boolean=True)
    idle = cp.Variable(T, boolean=True)

    # Done indicator
    done = cp.Variable(T, boolean=True)

    # One-hot position encoding
    pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

    constraints = []

    # Initial conditions
    constraints += [soc[0] == 6, budget[0] == 12, pos_bin[0, 0] == 1]
    for i in range(1, n_nodes):
        constraints += [pos_bin[0, i] == 0]

    R_bar = 6  # Charging cost

    for t in range(T):
        constraints += [cp.sum(pos_bin[t, :]) == 1, cp.sum(pos_bin[t + 1, :]) == 1]
        constraints += [move[t] + charge[t] + idle[t] == 1]

        # ---------------------------

        # We're having an issue with the movement constraints!!! They're allowing both charging and moving at the same time...

        # ---------------------------

        # Movement constraints
        for i in range(n_nodes - 1):
            constraints += [
                # Transition from node i to i+1 only if move[t] == 1 and charge[t] == 0
                pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
                pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0,

                # Can only move when you don't charge. But the constraints aren't working!!!
                pos_bin[t, i] + charge[t] - pos_bin[t + 1, i + 1] <= 1,
                pos_bin[t + 1, i + 1] - pos_bin[t, i] - charge[t] <= 0,
            ]
        constraints += [pos_bin[t, 4] + move[t] <= 1]  # Can't move past node 4

        # Dynamics
        constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
        constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
        constraints += [soc[t + 1] >= 4, budget[t + 1] >= 0]
        constraints += [charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx]]

        # Lock state if idle
        constraints += [soc[t + 1] - soc[t] <= M * (1 - idle[t])]
        constraints += [soc[t] - soc[t + 1] <= M * (1 - idle[t])]
        constraints += [budget[t + 1] - budget[t] <= M * (1 - idle[t])]
        constraints += [budget[t] - budget[t + 1] <= M * (1 - idle[t])]
        for i in range(n_nodes):
            constraints += [
                pos_bin[t + 1, i] - pos_bin[t, i] <= M * (1 - idle[t]),
                pos_bin[t, i] - pos_bin[t + 1, i] <= M * (1 - idle[t])
            ]

        # Done flag if at destination
        constraints += [done[t] <= pos_bin[t, n_nodes - 1]]

    # Final conditions
    constraints += [pos_bin[T, 4] == 1, soc[T] >= 5]

    # Maximize early completion
    objective = cp.Maximize(cp.sum(idle))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS_BB)

    if soc.value is None:
        return None, None, None, None, None, None

    result = {
        "t": list(range(T + 1)),
        "soc": soc.value.round(2),
        "budget": budget.value.round(2),
        "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
        "move": list(move.value.round().astype(int)) + [None],
        "charge": list(charge.value.round().astype(int)) + [None],
        "idle": list(idle.value.round().astype(int)) + [None],
        "done": list(done.value.round().astype(int)) + [None],
    }

    df = pd.DataFrame(result).set_index("t")
    display(df)
    return soc.value, budget.value, move.value, charge.value, idle.value, pos_bin.value

# Example call (with sample charger positions):
solve_vehicle_problem_with_idle_and_done(3, 4)


(None, None, None, None, None, None)

In [ ]:
def solve_ev_feasibility(q1_idx, q2_idx, mip_constraints):
    soc, budget, move, charge, idle, pos_bin = solve_vehicle_problem_with_idle_and_done(q1_idx, q2_idx)
    all_vars = [soc, budget, pos_bin, move, charge, idle]
    feasible = all(x is not None for x in all_vars)
    return {
        'problem': 0 if feasible else 1,
        'solvertime': 0.0,
        'info': 'EV deterministic MILP feasibility'
    }


solve_ev_feasibility(3, 4)

{'problem': 1, 'solvertime': 0.0, 'info': 'EV deterministic MILP feasibility'}

In [39]:
# import cvxpy as cp
# import numpy as np
# import pandas as pd

# def solve_vehicle_problem_with_position_1D(q1_idx, q2_idx):
#     T = 6
#     n_nodes = 5
#     M = 1000

#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)
#     position = cp.Variable(T + 1, integer=True)

#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     idle = cp.Variable(T, boolean=True)
#     done = cp.Variable(T, boolean=True)

#     constraints = [
#         soc[0] == 6,
#         budget[0] == 12,
#         position[0] == 0,
#     ]

#     R_bar = 6

#     for t in range(T):
#         constraints += [
#             move[t] + charge[t] + idle[t] == 1,
#             position[t + 1] == position[t] + move[t],
#             position[t + 1] <= position[t] + (1 - charge[t]) * M,
#             position[t + 1] >= position[t] - (1 - charge[t]) * M,
#             position[t + 1] <= n_nodes - 1,
#             position[t + 1] >= 0,
#             soc[t + 1] == soc[t] + 2 * charge[t] - move[t],
#             budget[t + 1] == budget[t] - R_bar * charge[t],
#             soc[t + 1] >= 4,
#             budget[t + 1] >= 0,
#             charge[t] <= (q1_idx) * (position[t] == q1_idx) + (q2_idx) * (position[t] == q2_idx),
#             done[t] <= (position[t] == n_nodes - 1),
#             soc[t + 1] - soc[t] <= M * (1 - idle[t]),
#             soc[t] - soc[t + 1] <= M * (1 - idle[t]),
#             budget[t + 1] - budget[t] <= M * (1 - idle[t]),
#             budget[t] - budget[t + 1] <= M * (1 - idle[t]),
#             position[t + 1] - position[t] <= M * (1 - idle[t]),
#             position[t] - position[t + 1] <= M * (1 - idle[t]),
#         ]

#     constraints += [
#         position[T] == n_nodes - 1,
#         soc[T] >= 5
#     ]

#     objective = cp.Maximize(cp.sum(idle))
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         print("No feasible solution found.")
#         return None

#     df = pd.DataFrame({
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": position.value.round(2),
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None],
#         "idle": list(idle.value.round().astype(int)) + [None],
#         "done": list(done.value.round().astype(int)) + [None],
#     }).set_index("t")

#     print(df)
#     return soc.value, budget.value, move.value, charge.value, idle.value, position.value

# solve_vehicle_problem_with_position_1D(1, 4)


In [43]:
# import cvxpy as cp
# import numpy as np
# import pandas as pd
# from IPython.display import display

# def solve_vehicle_problem_with_strict_movement(q1_idx, q2_idx):
#     T = 6  # Total time steps
#     n_nodes = 5
#     M = 1000  # Big-M constant

#     # State variables
#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)

#     # Binary action variables
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     idle = cp.Variable(T, boolean=True)

#     # Done indicator
#     done = cp.Variable(T, boolean=True)

#     # One-hot position encoding
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = []

#     # Initial conditions
#     constraints += [soc[0] == 6, budget[0] == 12, pos_bin[0, 0] == 1]
#     for i in range(1, n_nodes):
#         constraints += [pos_bin[0, i] == 0]

#     R_bar = 6  # Charging cost

#     for t in range(T):
#         constraints += [cp.sum(pos_bin[t, :]) == 1, cp.sum(pos_bin[t + 1, :]) == 1]
#         constraints += [move[t] + charge[t] + idle[t] == 1]

#         # Prevent position change unless move[t] == 1
#         for i in range(n_nodes):
#             constraints += [
#                 pos_bin[t + 1, i] >= pos_bin[t, i] - move[t],
#                 pos_bin[t + 1, i] <= pos_bin[t, i] + (1 - move[t])
#             ]

#         # Allow move from i to i+1 only if move[t] == 1
#         for i in range(n_nodes - 1):
#             constraints += [
#                 pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
#                 pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0
#             ]
#         constraints += [pos_bin[t, 4] + move[t] <= 1]  # Can't move past node 4

#         # Dynamics
#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
#         constraints += [soc[t + 1] >= 4, budget[t + 1] >= 0]
#         constraints += [charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx]]

#         # Lock state if idle
#         constraints += [soc[t + 1] - soc[t] <= M * (1 - idle[t])]
#         constraints += [soc[t] - soc[t + 1] <= M * (1 - idle[t])]
#         constraints += [budget[t + 1] - budget[t] <= M * (1 - idle[t])]
#         constraints += [budget[t] - budget[t + 1] <= M * (1 - idle[t])]
#         for i in range(n_nodes):
#             constraints += [
#                 pos_bin[t + 1, i] - pos_bin[t, i] <= M * (1 - idle[t]),
#                 pos_bin[t, i] - pos_bin[t + 1, i] <= M * (1 - idle[t])
#             ]

#         # Done flag if at destination
#         constraints += [done[t] <= pos_bin[t, n_nodes - 1]]

#     # Final conditions
#     constraints += [pos_bin[T, 4] == 1, soc[T] >= 5]

#     # Maximize early completion
#     objective = cp.Maximize(cp.sum(idle))
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None, None, None, None, None

#     result = {
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None],
#         "idle": list(idle.value.round().astype(int)) + [None],
#         "done": list(done.value.round().astype(int)) + [None],
#     }

#     df = pd.DataFrame(result).set_index("t")
#     display(df)
#     return soc.value, budget.value, move.value, charge.value, idle.value, pos_bin.value

# solve_vehicle_problem_with_strict_movement(0, 4)


In [30]:
# import cvxpy as cp
# import numpy as np
# import pandas as pd

# def solve_vehicle_problem_with_done_lock(q1_idx, q2_idx):
#     T = 6  # Planning horizon
#     n_nodes = 5

#     # Variables
#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     done = cp.Variable(T, boolean=True)
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = []

#     # Initial state
#     constraints += [
#         soc[0] == 6,
#         budget[0] == 12,
#         pos_bin[0, 0] == 1,
#         *[pos_bin[0, i] == 0 for i in range(1, n_nodes)]
#     ]

#     R_bar = 6  # Charging cost
#     M = 1000   # Big-M constant for logic encoding

#     for t in range(T):
#         # One-hot encoding for positions
#         constraints += [cp.sum(pos_bin[t, :]) == 1, cp.sum(pos_bin[t + 1, :]) == 1]

#         # Movement logic (only forward by 1)
#         for i in range(n_nodes - 1):
#             constraints += [
#                 pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
#                 pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0,
#             ]
#         constraints += [pos_bin[t, 4] + move[t] <= 1]  # cannot move from node 4

#         # Standard dynamics
#         constraints += [
#             soc[t + 1] == soc[t] + 2 * charge[t] - move[t],
#             budget[t + 1] == budget[t] - R_bar * charge[t],
#             move[t] + charge[t] + done[t] == 1,
#             soc[t + 1] >= 4,
#             budget[t + 1] >= 0,
#             charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx],
#         ]

#         # Enforce lock in done state
#         for i in range(n_nodes):
#             constraints += [
#                 pos_bin[t + 1, i] - pos_bin[t, i] <= M * (1 - done[t]),
#                 pos_bin[t, i] - pos_bin[t + 1, i] <= M * (1 - done[t]),
#             ]
#         constraints += [
#             soc[t + 1] - soc[t] <= M * (1 - done[t]),
#             soc[t] - soc[t + 1] <= M * (1 - done[t]),
#             budget[t + 1] - budget[t] <= M * (1 - done[t]),
#             budget[t] - budget[t + 1] <= M * (1 - done[t]),
#         ]

#     # End at node 4 with sufficient charge
#     constraints += [pos_bin[T, 4] == 1, soc[T] >= 5]

#     # Maximize time spent in done state (encourage early completion)
#     objective = cp.Maximize(cp.sum(done))
#     prob = cp.Problem(objective, constraints)
#     prob.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None

#     df = pd.DataFrame({
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None],
#         "done": list(done.value.round().astype(int)) + [None],
#     }).set_index("t")

#     display("Vehicle State with Done Lock", df)

#     return df

# solve_vehicle_problem_with_done_lock(0, 4)


In [31]:
# import cvxpy as cp
# import numpy as np
# import pandas as pd

# def solve_vehicle_problem_with_done_trigger(q1_idx, q2_idx):
#     T = 6
#     n_nodes = 5

#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     done = cp.Variable(T, boolean=True)
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = []

#     constraints += [soc[0] == 6]
#     constraints += [budget[0] == 12]
#     constraints += [pos_bin[0, 0] == 1]
#     for i in range(1, n_nodes):
#         constraints += [pos_bin[0, i] == 0]

#     R_bar = 6
#     M = 1000

#     for t in range(T):
#         constraints += [cp.sum(pos_bin[t, :]) == 1]
#         constraints += [cp.sum(pos_bin[t + 1, :]) == 1]

#         for i in range(n_nodes - 1):
#             constraints += [
#                 pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
#                 pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0
#             ]
#         constraints += [pos_bin[t, 4] + move[t] <= 1]

#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
#         constraints += [move[t] + charge[t] + done[t] == 1]
#         constraints += [soc[t + 1] >= 4]
#         constraints += [budget[t + 1] >= 0]
#         constraints += [charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx]]

#         # done[t] is true only if vehicle at final node at time t
#         constraints += [done[t] <= pos_bin[t, n_nodes - 1]]

#         # If done, lock state values (remain unchanged)
#         constraints += [soc[t + 1] - soc[t] <= M * (1 - done[t])]
#         constraints += [soc[t] - soc[t + 1] <= M * (1 - done[t])]
#         constraints += [budget[t + 1] - budget[t] <= M * (1 - done[t])]
#         constraints += [budget[t] - budget[t + 1] <= M * (1 - done[t])]
#         for i in range(n_nodes):
#             constraints += [pos_bin[t + 1, i] - pos_bin[t, i] <= M * (1 - done[t])]
#             constraints += [pos_bin[t, i] - pos_bin[t + 1, i] <= M * (1 - done[t])]

#     constraints += [pos_bin[T, 4] == 1]
#     constraints += [soc[T] >= 5]

#     objective = cp.Maximize(cp.sum(done))
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None

#     df = pd.DataFrame({
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None],
#         "done": list(done.value.round().astype(int)) + [None]
#     }).set_index("t")

#     display("Vehicle State with Done Trigger", df)

#     return df

# solve_vehicle_problem_with_done_trigger(1, 4)


In [32]:
# import cvxpy as cp
# import numpy as np
# import pandas as pd
# from IPython.display import display

# def solve_vehicle_problem_fastest_completion(q1_idx, q2_idx):
#     T = 6  # Total time steps
#     n_nodes = 5

#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
#     done = cp.Variable(T, boolean=True)
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = [
#         soc[0] == 6,
#         budget[0] == 12,
#         pos_bin[0, 0] == 1
#     ]
#     for i in range(1, n_nodes):
#         constraints.append(pos_bin[0, i] == 0)

#     R_bar = 6

#     for t in range(T):
#         constraints.append(cp.sum(pos_bin[t, :]) == 1)
#         constraints.append(cp.sum(pos_bin[t + 1, :]) == 1)

#         for i in range(n_nodes - 1):
#             constraints.append(pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1)
#             constraints.append(pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0)

#         constraints.append(pos_bin[t, 4] + move[t] <= 1)

#         constraints.append(soc[t + 1] == soc[t] + 2 * charge[t] - move[t])
#         constraints.append(budget[t + 1] == budget[t] - R_bar * charge[t])
#         constraints.append(move[t] + charge[t] + done[t] == 1)
#         constraints.append(soc[t + 1] >= 4)
#         constraints.append(budget[t + 1] >= 0)
#         constraints.append(charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx])

#         # Link done[t] to terminal state detection
#         constraints.append(done[t] <= pos_bin[t, 4])
#         constraints.append(done[t] <= soc[t] / 5.0)
#         constraints.append(done[t] <= budget[t] / 0.01)  # force done only if feasible

#     # Ensure done is triggered exactly once (only one timestep where we mark trip complete)
#     # constraints.append(cp.sum(done) == 1)

#     objective = cp.Maximize(cp.sum(done))  # Encourages earliest done[t] = 1
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None, None, None, None, None

#     result = {
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None]
#     }

#     df = pd.DataFrame(result).set_index("t")
#     display("Vehicle State Trajectory", df)
#     return soc.value, budget.value, move.value, charge.value, pos_bin.value

# # Run example
# solve_vehicle_problem_fastest_completion(1, 4)


In [33]:
# # vehicle_planner.py
# import cvxpy as cp
# import numpy as np

# def solve_vehicle_problem(q1_idx, q2_idx):
#     # Vehicle state variables over 5 time steps (0 to 4)
#     T = 5
#     soc = cp.Variable(T+1) # Should be length T+1 because we make a decision at each node {0, 1, ... T-1}. We care about the final state at time T.
#     pos = cp.Variable(T+1, integer=True)
#     budget = cp.Variable(T+1)

#     # Binary decision: move or charge at each step (mutually exclusive)
#     move = cp.Variable(T, boolean=True) # Move or charge at each of {0, 1, ... T-1}.
#     charge = cp.Variable(T, boolean=True)

#     constraints = []

#     # Initial conditions
#     constraints += [soc[0] == 6]
#     constraints += [pos[0] == 0]
#     constraints += [budget[0] == 12]

#     R_bar = 6  # fixed charge cost rate

#     M = 1000  # big-M constant

#     for t in range(T):
#         # Dynamics
#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]] # Increase soc if we charge or decrease if we move.
#         constraints += [pos[t + 1] == pos[t] + move[t]] # Update position to reflect whether or not we move
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]] # Update budget to reflect whether or not we charged
#         constraints += [move[t] + charge[t] == 1] # Indicates that we either charge or move
#         constraints += [soc[t + 1] >= 4] # Embed that soc must always stay above 4
#         constraints += [budget[t + 1] >= 0] # Encode that budget must always be positive
        
#         # Experiment
#         # --- New: binary indicator for being at a charger ---
#         at_charger = cp.Variable(boolean=True)

#         # Define when pos[t] == q1_idx or q2_idx
#         # Introduce constraints to "activate" at_charger only at charger locations
#         constraints += [pos[t] - q1_idx <= M * (1 - at_charger)]
#         constraints += [q1_idx - pos[t] <= M * (1 - at_charger)]
#         constraints += [pos[t] - q2_idx <= M * (1 - at_charger)]
#         constraints += [q2_idx - pos[t] <= M * (1 - at_charger)]

#         # Only allow charging at charger locations
#         constraints += [charge[t] <= at_charger]

#     # Final conditions
#     constraints += [pos[T] == 4] # Car must reach final node
#     constraints += [soc[T] >= 5] # Car must end with at least 5 units of energy

#     objective = cp.Minimize(cp.sum(charge))  # minimize charges (can adjust later)
#     problem = cp.Problem(objective, constraints)
#     problem.solve()

#     return soc.value, pos.value, budget.value, move.value, charge.value

In [34]:
# # Re-import necessary packages after code execution state reset
# import cvxpy as cp
# import numpy as np

# def solve_vehicle_problem_debug(q1_idx, q2_idx):
#     T = 6  # Need T=6 steps for your expected behavior (0 to 6), you had only T=5
#     soc = cp.Variable(T + 1)
#     pos = cp.Variable(T + 1, integer=True)
#     budget = cp.Variable(T + 1)

#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)

#     constraints = [
#         soc[0] == 6,
#         pos[0] == 0,
#         budget[0] == 12,
#     ]

#     R_bar = 6
#     M = 1000

#     for t in range(T):
#         # Dynamics
#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
#         constraints += [pos[t + 1] == pos[t] + move[t]]
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
#         constraints += [move[t] + charge[t] == 1]
#         constraints += [soc[t + 1] >= 4]
#         constraints += [budget[t + 1] >= 0]

#         # Binary indicators for charger presence
#         at_q1 = cp.Variable(boolean=True)
#         at_q2 = cp.Variable(boolean=True)
#         at_charger = cp.Variable(boolean=True)

#         # Big-M trick to encode pos[t] == q1_idx or q2_idx
#         constraints += [pos[t] - q1_idx <= M * (1 - at_q1)]
#         constraints += [q1_idx - pos[t] <= M * (1 - at_q1)]
#         constraints += [pos[t] - q2_idx <= M * (1 - at_q2)]
#         constraints += [q2_idx - pos[t] <= M * (1 - at_q2)]

#         # OR logic: at_charger = at_q1 OR at_q2
#         constraints += [at_charger >= at_q1]
#         constraints += [at_charger >= at_q2]
#         constraints += [at_charger <= at_q1 + at_q2]

#         # Only allow charging if at a charger
#         constraints += [charge[t] <= at_charger]

#     constraints += [
#         pos[T] == 4,
#         soc[T] >= 5,
#     ]

#     problem = cp.Problem(cp.Minimize(cp.sum(charge)), constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     return soc.value, pos.value, budget.value, move.value, charge.value

# # Run with q1_idx = 0 and q2_idx = 4 as you described
# solve_vehicle_problem_debug(0, 4)


In [35]:
# import cvxpy as cp
# import numpy as np
# # from ace_tools import display_dataframe_to_user
# from IPython.display import display

# def solve_vehicle_problem_binary_position(q1_idx, q2_idx):
#     T = 6  # Total time steps
#     n_nodes = 5

#     # State variables
#     soc = cp.Variable(T + 1)
#     budget = cp.Variable(T + 1)

#     # Binary variables
#     move = cp.Variable(T, boolean=True)
#     charge = cp.Variable(T, boolean=True)
    
#     # One-hot position encoding: pos_bin[t][i] = 1 if at node i at time t
#     pos_bin = cp.Variable((T + 1, n_nodes), boolean=True)

#     constraints = []

#     # Initial conditions
#     constraints += [soc[0] == 6]
#     constraints += [budget[0] == 12]
#     constraints += [pos_bin[0, 0] == 1]  # Start at node 0
#     for i in range(1, n_nodes):
#         constraints += [pos_bin[0, i] == 0]

#     R_bar = 6  # Charging cost

#     for t in range(T):
#         # Ensure exactly one position at each time
#         constraints += [cp.sum(pos_bin[t, :]) == 1]
#         constraints += [cp.sum(pos_bin[t + 1, :]) == 1]

#         # Movement constraints: only forward by 1
#         for i in range(n_nodes - 1):
#             constraints += [
#                 pos_bin[t, i] + move[t] - pos_bin[t + 1, i + 1] <= 1,
#                 pos_bin[t + 1, i + 1] - pos_bin[t, i] - move[t] <= 0
#             ]
#         # Ensure move from node 4 is not allowed
#         constraints += [pos_bin[t, 4] + move[t] <= 1]

#         # Dynamics
#         constraints += [soc[t + 1] == soc[t] + 2 * charge[t] - move[t]]
#         constraints += [budget[t + 1] == budget[t] - R_bar * charge[t]]
#         constraints += [move[t] + charge[t] == 1]
#         constraints += [soc[t + 1] >= 4]
#         constraints += [budget[t + 1] >= 0]

#         # Charging only at charger nodes (q1 and q2)
#         constraints += [charge[t] <= pos_bin[t, q1_idx] + pos_bin[t, q2_idx]]

#     # Final conditions
#     constraints += [pos_bin[T, 4] == 1]  # Must end at node 4
#     constraints += [soc[T] >= 5]

#     # Define and solve problem
#     objective = cp.Minimize(cp.sum(charge))
#     problem = cp.Problem(objective, constraints)
#     problem.solve(solver=cp.ECOS_BB)

#     if soc.value is None:
#         return None, None, None, None, None

#     result = {
#         "t": list(range(T + 1)),
#         "soc": soc.value.round(2),
#         "budget": budget.value.round(2),
#         "position": [np.argmax(pos_bin.value[t]) for t in range(T + 1)],
#         "move": list(move.value.round().astype(int)) + [None],
#         "charge": list(charge.value.round().astype(int)) + [None]
#     }

#     import pandas as pd
#     df = pd.DataFrame(result)
#     display("Vehicle State Trajectory", df)
#     return soc.value, budget.value, move.value, charge.value, pos_bin.value

# # Example run
# solve_vehicle_problem_binary_position(0, 4)


In [36]:
# budget.value

In [37]:
# solve_vehicle_problem(q1_idx, q2_idx)

In [38]:
# # contract_utils.py
# def guarantee_based_on(b, p):
#     # Compose a temporal logic string (mock version)
#     charger_indices = [i for i, val in enumerate(b) if val == 1]

#     if len(charger_indices) != 2:
#         raise ValueError("Exactly two chargers must be installed.")

#     q1, q2 = sorted(charger_indices)

#     g = []
#     g.append(f"AP(1): q1 <= q2")  # Order
#     g.append(f"AP(2): cost(q1, q2) = {q1 + q2} <= 7")  # Charger cost
#     g.append("AP(3): vehicle soc >= 4 always")
#     g.append("AP(4): vehicle soc >= 5 at final")
#     g.append("AP(5): vehicle reaches final node")
#     g.append("AP(6): budget always >= 0")
#     g.append("AP(7): profit >= 0")
#     return " and ".join(g)

In [39]:
# # cno_optimization.py
# import cvxpy as cp
# import numpy as np

# def solve_cno_problem():
#     # Network has 5 nodes: 0 through 4
#     n_nodes = 5
#     node_indices = np.arange(n_nodes)

#     # Binary variables: b[i] = 1 if charger is installed at node i
#     b = cp.Variable(n_nodes, boolean=True)

#     # Price variables (only active if b[i] = 1)
#     # p = cp.Variable(n_nodes)
    
#     # Integer indices for the charger locations
#     q1_idx = cp.Variable(integer=True)
#     q2_idx = cp.Variable(integer=True)

#     # Constraint
#     constraints = [
#         cp.sum(b) == 2, # Exactly two chargers installed
#         q1_idx >= 0, q1_idx <= 4, # Charger place on one of the 5 nodes
#         q2_idx >= 0, q2_idx <= 4, # Charger place on one of the 5 nodes

#         # q1_idx == 0,# Test 

#         q1_idx + q2_idx == cp.sum(cp.multiply(b, node_indices))  # The sum of the charger indices is consistent with 
#     ]

#     # Define price pi = (3-i)*qi_idx
#     p1 = (3-1)*q1_idx
#     p2 = (3-2)*q2_idx

#     # Modified cost to encourage spread between q1 and q2
#     # Encourage larger spread: penalize closeness → reward larger distance
#     cost = (q1_idx + q2_idx) - 1.5 * (q2_idx - q1_idx)

#     # Keep cost in valid range and ensure profitability
#     constraints += [
#         cost >= 0,
#         cost <= 7,
#         q1_idx <= q2_idx  # enforce ordering: q1 <= q2
#     ]

#     # Simple revenue model: total of active prices
#     revenue = p1 + p2
#     profit = revenue - cost
#     constraints.append(profit >= 0)  # Ensure profit is non-negative

#     # Define and solve the optimization problem
#     objective = cp.Maximize(profit)
#     prob = cp.Problem(objective, constraints)
#     prob.solve()

#     print("Optimization status:", prob.status)
#     print("q1_idx + q2_idx:", q1_idx.value + q2_idx.value)
#     print("cost (encouraging spread):", cost.value)
#     print("revenue:", revenue.value)
#     print("profit:", profit.value)

#     return b.value.round().astype(int), q1_idx.value, q2_idx.value